In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure, restoration

import cv2 as cv

In [ ]:
image_path = 'data/original/1.pgm'
image_groundtruth_path = 'data/groundtruth/1_gt.pgm'

In [ ]:
image = plt.imread(image_path)
image_groundtruth = plt.imread(image_groundtruth_path)

In [ ]:
image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

## Pre procesamiento

In [ ]:
def equalize(image):
    return cv.equalizeHist(image)

In [ ]:
def inpaiting(image, mask):
    return cv.inpaint(image, mask, inpaintRadius=3, flags=cv.INPAINT_TELEA)

In [ ]:
equalized = equalize(image)
mask = np.zeros(image.shape, dtype=np.uint8)

In [ ]:
inpaint = inpaiting(equalized, mask)


In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.title('Original')
plt.imshow(image, cmap='gray')
plt.subplot(1, 3, 2)
plt.title('Ecualizada')
plt.imshow(equalized, cmap='gray')
plt.subplot(1, 3, 3)
plt.title('Inpainting')
plt.imshow(inpaint, cmap='gray')
plt.show()

## Niblack

In [ ]:
def niblack_thresholding(image, window_size=15, k=-0.2):
    x, y = image.shape
    threshold = np.zeros((x, y), dtype=np.uint8)
    
    for i in range(x):
        for j in range(y):
            x1 = max(0, i - window_size // 2)
            x2 = min(x, i + window_size // 2)
            y1 = max(0, j - window_size // 2)
            y2 = min(y, j + window_size // 2)
            
            window = image[x1:x2, y1:y2]
            mean = np.mean(window)
            std = np.std(window)
            
            threshold[i, j] = mean - k * std
            
    return threshold

In [ ]:
threshold = niblack_thresholding(image, window_size=21, k=0.2)
binary = image < threshold

In [ ]:
def gaussian_blur(image, kernel_size=5, sigma=1.0):
    ax = np.linspace(-(kernel_size - 1) / 2., (kernel_size - 1) / 2., kernel_size)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sigma))
    kernel = np.outer(gauss, gauss)
    kernel = kernel / np.sum(kernel)
    
    padded_image = np.pad(image, kernel_size // 2, mode='reflect')
    blurred = np.zeros_like(image)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i+kernel_size, j:j+kernel_size]
            blurred[i, j] = np.sum(region * kernel)
    
    return blurred

In [ ]:
def sobel_edge_detection(image):
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    
    padded_image = np.pad(image, 1, mode='reflect')
    grad_x = np.zeros_like(image)
    grad_y = np.zeros_like(image)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i+3, j:j+3]
            grad_x[i, j] = np.sum(region * sobel_x)
            grad_y[i, j] = np.sum(region * sobel_y)
    
    edges = np.sqrt(grad_x**2 + grad_y**2)
    edges = (edges / edges.max() * 255).astype(np.uint8)
    
    return edges

In [ ]:
def dilate(image, kernel_size=3):
    kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    padded_image = np.pad(image, kernel_size // 2, mode='constant', constant_values=0)
    dilated = np.zeros_like(image)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i+kernel_size, j:j+kernel_size]
            dilated[i, j] = np.max(region * kernel)
    
    return dilated

In [ ]:
def median_filter(image, kernel_size=3):
    padded_image = np.pad(image, kernel_size // 2, mode='reflect')
    filtered = np.zeros_like(image)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i+kernel_size, j:j+kernel_size]
            filtered[i, j] = np.median(region)
    
    return filtered

In [ ]:
def label(image):
    labeled_image = np.zeros_like(image, dtype=np.int32)
    label_count = 1
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i, j] == 255 and labeled_image[i, j] == 0:
                stack = [(i, j)]
                while stack:
                    x, y = stack.pop()
                    if labeled_image[x, y] == 0:
                        labeled_image[x, y] = label_count
                        for dx in [-1, 0, 1]:
                            for dy in [-1, 0, 1]:
                                nx, ny = x + dx, y + dy
                                if 0 <= nx < image.shape[0] and 0 <= ny < image.shape[1]:
                                    if image[nx, ny] == 255 and labeled_image[nx, ny] == 0:
                                        stack.append((nx, ny))
                label_count += 1
    
    return labeled_image, label_count - 1


In [ ]:
def remove_small_objects(binary_image, min_size=150):
    labeled_image, num_labels = label(binary_image)
    sizes = np.bincount(labeled_image.ravel())
    mask_sizes = sizes >= min_size
    mask_sizes[0] = 0  # Eliminar el fondo
    
    cleaned_image = mask_sizes[labeled_image]
    return cleaned_image.astype(np.uint8) * 255


In [ ]:
def binarize_arteries(image):
    blurred = gaussian_blur(image)
    edges = sobel_edge_detection(blurred)
    dilated = dilate(edges)
    median_filtered = median_filter(dilated)
    binary_image = 255 - median_filtered
    cleaned_image = remove_small_objects(binary_image)
    return cleaned_image

In [ ]:
binary = binarize_arteries(image)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original')
plt.imshow(image, cmap='gray')
plt.subplot(1, 2, 2)
plt.title('Binarizada')
plt.imshow(binary, cmap='gray')
plt.show()

## Post procesamiento

## Resultados 

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(1, 2, 1)
plt.imshow(binary, cmap='gray')
plt.title('Binary')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(image_groundtruth, cmap='gray')
plt.title('Groundtruth')
plt.axis('off')

plt.show()

## Métricas